In [1]:
import sys 
sys.path.append("..")

In [3]:
import numpy as np
from ppe_tools import Ensemble,Member,ParamInfo
from ppe_tools.utils import get_default, parse_val
import xarray as xr
import os
import pandas as pd
import glob

In [4]:
## download the google spreadsheet
data_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQs413GtLXtHVDCqEPgAwn4BbDjoWmV7uFqOAWH4mgpxXoVfN6ijnJdhyRgLkV-n2eU-sSQush4CzYU/pub?output=csv'
cmd = 'curl -L '+data_url+' > sept28.csv' # need to add -L option to force redirects
os.system(cmd)

0

In [5]:
csvfile = 'sept28.csv'
data = pd.read_csv(csvfile,header=0,skiprows=[1]) # modify read_csv to account for header spanning 2 rows

In [7]:
pdir = '/glade/scratch/djk2120/PPEn11/paramfiles/'
last_oaat = sorted(glob.glob(pdir+'*OAAT*'))[-1]
nextnum = int(last_oaat.split('OAAT')[1].split('.')[0])+1
nextnum

462

In [8]:
#create dict for the new parameters
oaats = {}
names = ['flnr','dbh','leaf_mr_vcm','fnitr']
for name in names:
    
    #access data from csv
    ix = data['name']==name
    minval = data['min'][ix].values[0]
    maxval = data['max'][ix].values[0]
    pftmin = data['pft_mins'][ix].values[0]
    pftmax = data['pft_maxs'][ix].values[0]
    thisloc = data['location'][ix].values[0]

    #format the minimum
    needs_pft = (minval=='pft')
    if needs_pft:
        thismin = np.fromstring(pftmin, dtype='float', sep=',')
    elif 'percent' in minval:
        thismin = minval
    else:
        thismin = np.array(float(minval))

    #format the maximum
    needs_pft = (maxval=='pft')
    if needs_pft:
        thismax = np.fromstring(pftmax, dtype='float', sep=',')
    elif 'percent' in maxval:
        thismax = maxval
    else:
        thismax = np.array(float(maxval))

    #save in dictionary
    oaats[name]={'min':thismin,'max':thismax,'loc':thisloc}

In [9]:
oaats

{'flnr': {'min': '20percent', 'max': '20percent', 'loc': 'P'},
 'dbh': {'min': array([0.   , 0.1  , 0.1  , 0.1  , 0.1  , 0.1  , 0.1  , 0.1  , 0.1  ,
         0.05 , 0.05 , 0.05 , 0.008, 0.008, 0.008, 0.008, 0.008, 0.008,
         0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008,
         0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008,
         0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008,
         0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008,
         0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008,
         0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008,
         0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008]),
  'max': array([0.   , 0.6  , 0.6  , 0.6  , 0.6  , 0.6  , 0.6  , 0.6  , 0.6  ,
         0.1  , 0.1  , 0.1  , 0.008, 0.008, 0.008, 0.008, 0.008, 0.008,
         0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008,
         0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008,

In [10]:
#instantiate the Ensemble object
basefile = '/glade/p/cesm/cseg/inputdata/lnd/clm2/paramdata/ctsm51_params.c210507.nc'
pdir = '/glade/scratch/djk2120/PPEn11/paramfiles/'
ndir = '/glade/scratch/djk2120/PPEn11/namelist_mods/'
x    = Ensemble(basefile,pdir,ndir)

In [11]:
#add the new OAATS
prefix = 'OAAT'
x.add_oaats(oaats,prefix,nextnum,skipBFB=True)

In [14]:
#the bfb test is not correctly catching fnitr-max
#   removing it manually here
#   the issue is that it has a 0 for bare-ground, so I'll need to edit the bfb test to ignore bareground
x.remove_member(x.members[-1])

In [16]:
csvfile='/glade/scratch/djk2120/PPEn11/extras_for_sp.csv'
x.write(csvfile=csvfile)